In [1]:
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
import keras
import joblib
from keras.models import model_from_json
from sklearn.metrics import accuracy_score

2023-07-29 13:12:52.814140: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-29 13:12:53.062681: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-29 13:12:53.118448: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yonix42/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-07-29 13:12:53.118460: I

In [2]:
class LocalBinaryPatterns2:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius

    def describe(self, image, eps=1e-7):
        lbp = local_binary_pattern(
            image, self.numPoints, self.radius, method="nri_uniform")
        n_bins = int(lbp.max() + 1)
        (hist, _) = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))

        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        return hist

In [3]:
desc = LocalBinaryPatterns2(8, 2)

labels = []
images = []
X = []
y = []

path = './spec_evaluation_janela_completo/'

input_shape = (256, 256, 3)
classes = ["beach", "bus", "cafe_restaurant", "car", "city_center", "forest_path", "grocery_store",
           "home", "library", "metro_station", "office", "park", "residential_area", "train", "tram"]

max_imagens = 390
j = 0
k = 0
for i in range(max_imagens):
    if j == 26:
        j = 0
        k += 1
    path_total = path + 'class_' + classes[k] + '/'
    
    path_img = path_total + str(i) + '.png'
    img_rgb = cv2.imread(path_img)
    img_rgb = cv2.resize(img_rgb, (256, 256))
    images.append(img_rgb)

    img_gray = cv2.imread(path_img, 0)
    hist = desc.describe(img_gray)
    X.append(hist)

    labels.append(k)
    j += 1

y = labels[:]

In [4]:
model_path = './saida2/'

json_file = open(model_path+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights(model_path+'model.h5')

loaded_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

svm_model = joblib.load(model_path+'modelo_svm.pkl')


2023-07-29 13:13:50.019404: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yonix42/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-07-29 13:13:50.019657: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-29 13:13:50.019671: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (staub): /proc/driver/nvidia/version does not exist
2023-07-29 13:13:50.020212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/l

In [5]:
images = np.array(images)
predicoes_cnn = loaded_model.predict(images)
predicoes_svm = svm_model.predict_proba(X)
predicoes_real = y[:]

13/13 [==============================] - 7s 477ms/step


In [6]:
def borda_count(svm_preds, cnn_preds):
    bordas = np.abs(svm_preds - cnn_preds)

    return np.argmax(bordas, axis=1)

def soma_simples(svm_preds, cnn_preds):
    preds_compinadas = svm_preds + cnn_preds

    return np.argmax(preds_compinadas, axis=1)

def produto(svm_preds, cnn_preds):
    preds_compinadas = svm_preds * cnn_preds

    return np.argmax(preds_compinadas, axis=1)


def media_simples(svm_preds, cnn_preds):
    preds_combinadas = (svm_preds + cnn_preds) / 2.0

    return np.argmax(preds_combinadas, axis=1)

def media_ponderada(svm_preds, cnn_preds, peso_svm, peso_cnn):
    preds_combinadas = (peso_svm * svm_preds + peso_cnn *
                        cnn_preds) / (peso_svm + peso_cnn)

    return  np.argmax(preds_combinadas, axis=1)

def maiores_valores(svm_preds, cnn_preds):
    fusion_preds = np.maximum(svm_preds, cnn_preds)

    return  np.argmax(fusion_preds, axis=1)

def maioria_ponderada(svm_preds, cnn_preds, peso_svm, peso_cnn):
    fusion_preds = peso_svm * svm_preds + peso_cnn * cnn_preds

    classes_preditas = np.argmax(fusion_preds, axis=1)

    return classes_preditas

def fusao_com_rejeicao(svm_preds, cnn_preds, limiar_rejeicao=0.4):
    votos_totais = svm_preds + cnn_preds
    previsoes_finais = np.argmax(votos_totais, axis=1)
    confianca = np.max(votos_totais, axis=1)
    previsoes_finais[confianca < limiar_rejeicao] = -1  
    return previsoes_finais


In [7]:
nomes_metodos = ['Votação por Borda', 'Soma Simples', 'Produto', 'Média Simples',
                 'Média Ponderada', 'Maiores Valores', 'Maioria Ponderada', 
                 'Fusao com rejeicao']

lista_acuracia = np.array([])

preds_teste = borda_count(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_borda_count:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = soma_simples(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_soma_simples:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = produto(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_produto:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = media_simples(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_media_simples:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = media_ponderada(predicoes_svm, predicoes_cnn, 2, 8)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_media_ponderada:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = maiores_valores(predicoes_svm, predicoes_cnn)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_maiores_valores:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = maioria_ponderada(predicoes_svm, predicoes_cnn, 1, 2)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_maioria_ponderada:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

preds_teste = fusao_com_rejeicao(predicoes_svm, predicoes_cnn, limiar_rejeicao=0.8)
acc_aux = accuracy_score(predicoes_real, preds_teste)
print("acc_fusao_com_rejeicao:", acc_aux)
lista_acuracia = np.append(lista_acuracia, acc_aux)

melhor_metodo = np.argmax(lista_acuracia)
melhor_acuracia = lista_acuracia[melhor_metodo]
nome_melhor_metodo = nomes_metodos[melhor_metodo]

print(f'O melhor método é o {nome_melhor_metodo} com acuracia de {melhor_acuracia}')

acc_borda_count: 0.2641025641025641
acc_soma_simples: 0.2743589743589744
acc_produto: 0.3564102564102564
acc_media_simples: 0.2743589743589744
acc_media_ponderada: 0.6076923076923076
acc_maiores_valores: 0.2717948717948718
acc_maioria_ponderada: 0.5897435897435898
acc_fusao_com_rejeicao: 0.258974358974359
O melhor método é o Média Ponderada com acuracia de 0.6076923076923076
